In [132]:
import time
import bs4
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests

# --- selenium libraries
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By


In [142]:
def get_page_html_data_with_see_more_option(url, chromedriver_path="./chromedriver.exe", numbers_of_see_more_clicks=1, sleep_time=3):
    # Create selenium driver
    service = Service(chromedriver_path)
    driver = webdriver.Chrome(service=service)    
    driver.get(url)
    driver.maximize_window()

    for i in range(1,numbers_of_see_more_clicks+1):                
        button_xpath = "//button[@class='m-footer-loader--button m-component-footer--loader m-button']"                
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_time)
        driver.find_element(by=By.XPATH, value=button_xpath).click()                    
        time.sleep(sleep_time)
        
    html_data = driver.page_source
    driver.close()
    
    return html_data

In [143]:
def extract_hubpages_articles_from_HTML_data(html_data):        
    # Extract articles from HTML data 
    soup = BeautifulSoup(html_data, 'html.parser')    
    articles = soup.find_all("div", {"class": "m-card--content"})          
    
    return articles

In [144]:
def create_df_from_articles(articles, category):    
    # Create a lists for df
    categories = []
    headers = []
    contents= []

    for article in articles:
        categories.append(category)
        headers.append(article.find("h2").get_text())
        content = article.find("p", {"class": "m-card--body m-ellipsis--text"})
        if content is not None:
            contents.append(content.get_text())            
        else:
            contents.append(content)                
        
    return pd.DataFrame(
        {"category": categories,  
         "header": headers, 
         "contents": contents})

In [145]:
# "science", "weather" - doesnt exists
categories = ["technology","politics", "sports", "food", "entertainment", "business"]
for category in categories: 
    url = f"https://discover.hubpages.com/{category}/"
    html_data = get_page_html_data_with_see_more_option(url, numbers_of_see_more_clicks=1)
    articles = extract_hubpages_articles_from_HTML_data(html_data)
    df = create_df_from_articles(articles, category)
    
    print(df.shape)
    print(df)


Number of articles: 100
(100, 3)
      category                                             header  \
0   technology  5 Reasons Why the Fire TV Stick 4k Is the Top ...   
1   technology  Headphones On and World Off With Happy Plugs A...   
2   technology  Is Crypto Trading More Profitable Than Forex T...   
3   technology                Technology to Make Your Life Easier   
4   technology  Radiation Hates A Case If It’s SafeSleeve’s De...   
..         ...                                                ...   
95  technology  Top 5 Social Media Marketing Trends to Look Ou...   
96  technology                     Top 10 SmartPhones of All Time   
97  technology              Will Brain Chips Make Us Superhumans?   
98  technology             What Are the Best Smartphones in 2022?   
99  technology                5 New Gadgets to Keep on Your Radar   

                                             contents  
0   The Fire TV Stick 4K has quickly become the to...  
1   The Happy Plugs Air 1 